In [0]:
# from google.colab import files
# uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
DIRECTORY = "/content/drive/My Drive/Colab Notebooks/lsda/HW1/"

In [0]:
#!head /content/drive/My\ Drive/Colab\ Notebooks/lsda/HW1/data/sales_train_validation.csv
#!tail /content/drive/My\ Drive/Colab\ Notebooks/lsda/HW1/data/sell_prices.csv #11621
!tail /content/drive/My\ Drive/Colab\ Notebooks/lsda/HW1/data/calendar.csv #2016-06-19,11621,Sunday

2016-06-10,11619,Friday,7,6,2016,d_1960,,,,,1,0,0
2016-06-11,11620,Saturday,1,6,2016,d_1961,,,,,0,1,1
2016-06-12,11620,Sunday,2,6,2016,d_1962,,,,,0,1,1
2016-06-13,11620,Monday,3,6,2016,d_1963,,,,,0,1,0
2016-06-14,11620,Tuesday,4,6,2016,d_1964,,,,,0,0,1
2016-06-15,11620,Wednesday,5,6,2016,d_1965,,,,,0,1,1
2016-06-16,11620,Thursday,6,6,2016,d_1966,,,,,0,0,0
2016-06-17,11620,Friday,7,6,2016,d_1967,,,,,0,0,0
2016-06-18,11621,Saturday,1,6,2016,d_1968,,,,,0,0,0
2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [0]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

# Credits

* [First R notebook](https://www.kaggle.com/kailex/m5-forecaster-v2)
* [Python translation](https://www.kaggle.com/kneroma/m5-forecast-v2-python)

# Changes
* v5 : try to optimise the LGBM params (go below in lgbm params section to see changes)
* v4 : add df, X_train deletion before training step --> increasing train sample without memeroy issues

In [0]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [0]:
pd.options.display.max_columns = 50

In [0]:
h = 28 
max_lags = 57 #used in the final loop and for creating the te dataset
tr_last = 1913
fday = datetime(2016,4, 25) #FIRST DAY OF PREDICTION
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [0]:
#pd.read_csv(DIRECTORY + "data/sell_prices.csv", dtype = PRICE_DTYPES)
#pd.read_csv(DIRECTORY + "data/sales_train_validation.csv")

In [0]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    #load the sell prices data set and give each column the appropriate datatype
    prices = pd.read_csv(DIRECTORY + "data/sell_prices.csv", dtype = PRICE_DTYPES)
    #store_id, item_id, wm_yr_wk, sell_price
    for col, col_dtype in PRICE_DTYPES.items(): #for every column in the df (only 4 cols)
        if col_dtype == "category":
            #encode store_id and item_id as integer 1-7, 1-12 so on
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv(DIRECTORY + "data/calendar.csv", dtype = CAL_DTYPES)
    #date, wm_yr_wk, weekday, wday, event type..., snap_CA..WI
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category": 
            #encode as integer 1-7, 1-12 so on
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    # which days to read from csv
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    # which categories to read
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    #create a dictionary for all d_n and add the float32 dtype
    dtype = {numcol:"float32" for numcol in numcols} 
    #add the category columns and their corresponding d_type to the dictionary
    dtype.update({col: "category" for col in catcols if col != "id"})
    # usecols selects all the catgory columns and all the d_ columns we previously selected
    # norws = None selects all rows
    dt = pd.read_csv(DIRECTORY + "data/sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    #convert categorical varibles to integers
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    #when not training the model, convert last 28 day columns to NaNs
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    #merge all three data frames
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    #FINAL DTAFAME:
    #item_id...d.. event..sell_price
    #item1..day1..price
    #...
    #item1..day1900..price
    #item2..day1..price
    #...
    return dt

In [0]:
def create_fea(dt):
    #choose number of lags
    lags = [7, 28] 
    #create col names for each lag
    lag_cols = [f"lag_{lag}" for lag in lags ] 
    lag_cols = ["lag_7", "lag_28"]
    for lag, lag_col in zip(lags, lag_cols):
        #creates lag_cols filled with NaNs
        #supposed to shift the columns 7 down/28 down but doesnt really do anything here
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)
    
    wins = [7, 28] #creates two window sizes
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            # calculates the 7 and 28 day rolling mean, for both lag columns.
            # still results in columns filled with Nans so not sure why
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

        date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        #for every item in the features dict:
        if date_feat_name in dt.columns:
            #if the feature is in dt, convert it to an int16 
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
          #I think there'ssomesort of conversion between the date data type and the week, mnth, whatever output
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [0]:
FIRST_DAY = 1500 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [0]:
%%time
#load files, and merge to one dataframe
df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

CPU times: user 11.8 s, sys: 444 ms, total: 12.2 s
Wall time: 12.5 s


In [0]:
#df.loc[df.id == "HOUSEHOLD_2_516_WI_2_validation"]
#df.groupby("id").d.count()
#df.groupby("d").id.count().loc["d_1913"]
#df.isna().sum()

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12567124 entries, 0 to 12567123
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
dtypes: datetime64[ns](1), float32(5), int16(14), object(2)
memory us

In [0]:
%%time
#add lag_7	lag_28	rmean_7_7	rmean_28_7	rmean_7_28	rmean_28_28 (all NA)	week	quarter	mday columns
create_fea(df)
df.shape

CPU times: user 2min 11s, sys: 776 ms, total: 2min 12s
Wall time: 2min 12s


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12567124 entries, 0 to 12567123
Data columns (total 31 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
 22  lag_7         float32       
 23  lag_28        float32       
 

In [0]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1500,0.0,2015-03-08,11506,3,2,3,2015,0,0,0,0,1.0,0.0,1.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,10,1,8
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1501,0.0,2015-03-09,11506,1,3,3,2015,0,0,0,0,1.0,1.0,1.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,9
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1502,0.0,2015-03-10,11506,5,4,3,2015,0,0,0,0,1.0,0.0,0.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,10
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1503,1.0,2015-03-11,11506,6,5,3,2015,0,0,0,0,0.0,1.0,1.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,11
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1504,1.0,2015-03-12,11506,4,6,3,2015,0,0,0,0,0.0,1.0,1.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,12


In [0]:
#remove all rows with NA
df.dropna(inplace = True)
df.shape

(10890174, 31)

In [0]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
1650761,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1555,1.0,2015-05-02,11514,2,1,5,2015,0,0,0,0,1.0,0.0,1.0,8.26,2.0,2.0,0.857143,0.857143,0.750000,0.785714,18,2,2
1650762,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1556,2.0,2015-05-03,11514,3,2,5,2015,0,0,0,0,1.0,1.0,1.0,8.26,0.0,0.0,0.857143,0.571429,0.678571,0.785714,18,2,3
1650763,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1557,0.0,2015-05-04,11514,1,3,5,2015,0,0,0,0,1.0,0.0,0.0,8.26,1.0,0.0,0.857143,0.428571,0.678571,0.785714,19,2,4
1650764,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1558,0.0,2015-05-05,11514,5,4,5,2015,3,1,0,0,1.0,1.0,1.0,8.26,0.0,1.0,0.857143,0.571429,0.678571,0.821429,19,2,5
1650765,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1559,0.0,2015-05-06,11514,6,5,5,2015,0,0,0,0,1.0,1.0,1.0,8.26,0.0,2.0,0.571429,0.857143,0.678571,0.857143,19,2,6


In [0]:
#just ignore cat_feats for now
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
#For traning we use already have wday, month, year and item_id
#exclude everuthing in useless cols
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]
#############################
#PROGRESSED TILL HERE

In [0]:
X_train

,item_id,dept_id,store_id,cat_id,state_id,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
1650761,0,0,0,0,0,1,5,2015,0,0,0,0,1.0,0.0,1.0,8.26,2.0,2.0,0.857143,0.857143,0.750000,0.785714,18,2,2
1650762,0,0,0,0,0,2,5,2015,0,0,0,0,1.0,1.0,1.0,8.26,0.0,0.0,0.857143,0.571429,0.678571,0.785714,18,2,3
1650763,0,0,0,0,0,3,5,2015,0,0,0,0,1.0,0.0,0.0,8.26,1.0,0.0,0.857143,0.428571,0.678571,0.785714,19,2,4
1650764,0,0,0,0,0,4,5,2015,3,1,0,0,1.0,1.0,1.0,8.26,0.0,1.0,0.857143,0.571429,0.678571,0.821429,19,2,5
1650765,0,0,0,0,0,5,5,2015,0,0,0,0,1.0,1.0,1.0,8.26,0.0,2.0,0.571429,0.857143,0.678571,0.857143,19,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12567119,3046,6,9,2,2,2,4,2016,0,0,0,0,0.0,0.0,0.0,3.98,0.0,1.0,1.000000,0.714286,0.928571,1.250000,16,2,24
12567120,3047,6,9,2,2,1,4,2016,0,0,0,0,0.0,0.0,0.0,1.28,0.0,2.0,0.857143,1.142857,1.035714,1.107143,16,2,23
12567121,3047,6,9,2,2,2,4,2016,0,0,0,0,0.0,0.0,0.0,1.28,1.0,4.0,0.714286,1.571429,1.035714,1.250000,16,2,24
12567122,3048,6,9,2,2,1,4,2016,0,0,0,0,0.0,0.0,0.0,1.00,0.0,0.0,0.000000,2.285714,1.821429,1.785714,16,2,23


In [0]:
train_cols

Index(['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'wday', 'month',
       'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7', 'lag_28',
       'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28', 'week',
       'quarter', 'mday'],
      dtype='object')

In [0]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [0]:
%%time

np.random.seed(42)
#extracting 2M indices to use for valdiation data
fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False) 
#find difference between 2 sets, the differences are the training data
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
# convert training data to lgb dataset object
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
# convert validation data to lgb dataset object
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 4.13 s, sys: 11.9 ms, total: 4.14 s
Wall time: 4.14 s


In [0]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

43

In [0]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 120,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}


# params = {
#                     'boosting_type': 'gbdt',
#                     'objective': 'tweedie',
#                     'tweedie_variance_power': 1.1,
#                     'metric': 'rmse',
#                     'subsample': 0.5,
#                     'subsample_freq': 1,
#                     'learning_rate': 0.03,
#                     'num_leaves': 2**11-1,
#                     'min_data_in_leaf': 2**12-1,
#                     'feature_fraction': 0.5,
#                     'max_bin': 100,
#                     'n_estimators': 2,
#                     'boost_from_average': False,
#                     'verbose': -1,
#                 } 
VERSION = 6 #we remove non one hot coded parameters

In [0]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[20]	valid_0's rmse: 2.50262
[40]	valid_0's rmse: 2.22578
[60]	valid_0's rmse: 2.15594
[80]	valid_0's rmse: 2.13491
[100]	valid_0's rmse: 2.12569
[120]	valid_0's rmse: 2.12022
CPU times: user 10min 55s, sys: 1.43 s, total: 10min 56s
Wall time: 2min 56s


In [0]:
m_lgb.save_model(DIRECTORY + "models/model" + str(VERSION) + ".lgb")

In [0]:
alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]
    

In [0]:
create_dt(first_day=FIRST_DAY)

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1500,0.0,2015-03-08,11506,3,2,3,2015,0,0,0,0,1.0,0.0,1.0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1501,0.0,2015-03-09,11506,1,3,3,2015,0,0,0,0,1.0,1.0,1.0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1502,0.0,2015-03-10,11506,5,4,3,2015,0,0,0,0,1.0,0.0,0.0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1503,1.0,2015-03-11,11506,6,5,3,2015,0,0,0,0,0.0,1.0,1.0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1504,1.0,2015-03-12,11506,4,6,3,2015,0,0,0,0,0.0,1.0,1.0,8.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12567119,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1913,0.0,2016-04-24,11613,3,2,4,2016,0,0,0,0,0.0,0.0,0.0,3.98
12567120,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1912,1.0,2016-04-23,11613,2,1,4,2016,0,0,0,0,0.0,0.0,0.0,1.28
12567121,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1913,3.0,2016-04-24,11613,3,2,4,2016,0,0,0,0,0.0,0.0,0.0,1.28
12567122,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1912,0.0,2016-04-23,11613,2,1,4,2016,0,0,0,0,0.0,0.0,0.0,1.00


In [0]:
create_dt(False)

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1856,4.0,2016-02-27,11605,2,1,2,2016,0,0,0,0,0.0,0.0,0.0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1857,2.0,2016-02-28,11605,3,2,2,2016,0,0,0,0,0.0,0.0,0.0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1858,0.0,2016-02-29,11605,1,3,2,2016,0,0,0,0,0.0,0.0,0.0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1859,2.0,2016-03-01,11605,5,4,3,2016,0,0,0,0,1.0,1.0,0.0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1860,0.0,2016-03-02,11605,6,5,3,2016,0,0,0,0,1.0,0.0,1.0,8.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622135,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1941,NaN,2016-05-22,11617,3,2,5,2016,0,0,0,0,0.0,0.0,0.0,3.98
2622136,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1940,NaN,2016-05-21,11617,2,1,5,2016,0,0,0,0,0.0,0.0,0.0,1.28
2622137,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1941,NaN,2016-05-22,11617,3,2,5,2016,0,0,0,0,0.0,0.0,0.0,1.28
2622138,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1940,NaN,2016-05-21,11617,2,1,5,2016,0,0,0,0,0.0,0.0,0.0,1.00


In [0]:
alphas =[1.028] #[1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):
    #Data frame with the last 57 days
    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)] #F_1..F_28

    for tdelta in range(0, 28):
        #fday is the first day of prediction
        #create the date value for the 28 prediction dates
        day = fday + timedelta(days=tdelta)
        print(day)
        #print(tdelta, day)
        #we have a sliding window that copies data from the te dataset
        # we need this to calculate the lag features
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        #add lag features to the dataset
        create_fea(tst) 
        #select the day used for prediction
        tst = tst.loc[tst.date == day , train_cols]
        #fill th sales column with the prediction
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev

2016-04-25 00:00:00
1768420
2016-04-26 00:00:00
1768420
2016-04-27 00:00:00
1768420
2016-04-28 00:00:00
1768420
2016-04-29 00:00:00
1768420
2016-04-30 00:00:00
1768420
2016-05-01 00:00:00
1768420
2016-05-02 00:00:00
1768420
2016-05-03 00:00:00
1768420
2016-05-04 00:00:00
1768420
2016-05-05 00:00:00
1768420
2016-05-06 00:00:00
1768420
2016-05-07 00:00:00
1768420
2016-05-08 00:00:00
1768420
2016-05-09 00:00:00
1768420
2016-05-10 00:00:00
1768420
2016-05-11 00:00:00
1768420
2016-05-12 00:00:00
1768420
2016-05-13 00:00:00
1768420
2016-05-14 00:00:00
1768420
2016-05-15 00:00:00
1768420
2016-05-16 00:00:00
1768420
2016-05-17 00:00:00
1768420
2016-05-18 00:00:00
1768420
2016-05-19 00:00:00
1768420
2016-05-20 00:00:00
1768420
2016-05-21 00:00:00
1768420
2016-05-22 00:00:00
1768420


In [0]:
te.date 

0         2016-02-27
1         2016-02-28
2         2016-02-29
3         2016-03-01
4         2016-03-02
             ...    
2622135   2016-05-22
2622136   2016-05-21
2622137   2016-05-22
2622138   2016-05-21
2622139   2016-05-22
Name: date, Length: 2622140, dtype: datetime64[ns]

In [0]:
print(timedelta(1))

1 day, 0:00:00


In [0]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):
    #Data frame with the last 57 days
    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)] #F_1..F_28

    for tdelta in range(0, 28):
        #fday is the first day of prediction
        #create the date value for the 28 prediction dates
        day = fday + timedelta(days=tdelta)
        print(day)
        #print(tdelta, day)
        #we have a sliding window that copies data from the te dataset
        # we need this to calculate the lag features
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        #add lag features to the dataset
        create_fea(tst) 
        #select the day used for prediction
        tst = tst.loc[tst.date == day , train_cols]
        #fill th sales column with the prediction
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev


    #He uses 
    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    # if icount == 0 :
    #     sub = te_sub
    #     sub[cols] *= weight
    # else:
    #     sub[cols] += te_sub[cols]*weight
    # print(icount, alpha, weight)

sub = te_sub
sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv(DIRECTORY + "submissions/submission_v" + str(VERSION) + ".csv",index=False)

0 2016-04-25 00:00:00


KeyboardInterrupt: ignored

In [0]:
sub.head(10)

In [0]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

In [0]:
sub.shape

In [0]:
#Version 3: default, score: 0.49

In [0]:
# from google.colab import files
# files.download('m5/submissions/submissionv' + str(VERSION) + '.csv') 

In [0]:
# !ls
# !mv model.lgb m5/models/model.lgb
# !mv sample_submission.csv m5/data/sample_submission.csv
# !mv submissionv2.csv m5/submissions/submissionv2.csv

In [0]:
# !ls
# !wc -l submission*
# !wc -l m5/submissions/submissionv2.csv

In [0]:
# !tail m5/submissions/submissionv2.csv

In [0]:
#

#change num features from 1200 to 120
# change data from 350 to 750